In [ ]:
from datetime import datetime, timezone
import bs4 as bs
import os, shutil
import requests
import pandas as pd
from glob import glob
import numpy as np

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', None)

root_dir = os.path.dirname(os.getcwd()) + "/data/"

In [15]:
# Downloading file

file_type = "npy"

def download_files(folder_url, destination_folder, start_time, end_time, file_time):

  try:

    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
      os.makedirs(destination_folder)

    # Delete old files
    for filename in os.listdir(destination_folder):
      file_path = os.path.join(destination_folder, filename)
      try:
          if os.path.isfile(file_path) or os.path.islink(file_path):
              os.unlink(file_path)
          elif os.path.isdir(file_path):
              shutil.rmtree(file_path)
      except Exception as e:
          print('Failed to delete %s. Reason: %s' % (file_path, e))


    # Get the list of files in the folder
    response = requests.get(folder_url)
    data = bs.BeautifulSoup(response.text, "html.parser")

    csv_files = data.find_all("a", href=lambda href: href and href.endswith(f'.{file_type}'))

    # Filter CSV files based on time range
    filtered_files = []
    for file_name in csv_files:
        file_type_filename = file_name['href'].split('/')[-1]
        try:
            # Parse the filename to extract the creation or modification time
            file_time_str = file_type_filename.split('.')[0]  # Extract '20241021-031240'
            file_time = datetime.strptime(file_time_str, '%Y%m%d-%H%M%S')  # Use new format 'YYYYMMDD-HHMMSS'
            if start_time <= file_time <= end_time:
                filtered_files.append(file_name)
        except ValueError as e:
            # Handle parsing errors (e.g., invalid filename format)
            print(f"Error parsing filename: {file_type_filename}, {e.args[0]}")
    
    if len(filtered_files) == 0:
       print("No files matching that criteria")

    for file_name in filtered_files:
      file_type_url = file_name['href']  # Get the CSV file URL
      file_type_filename = file_type_url.split('/')[-1]  # Extract the filename
      link = folder_url + file_type_filename
      destination_link = os.path.join(destination_folder, file_type_filename)

      response = requests.get(link)
      with open(destination_link, 'wb') as f:
         f.write(response.content)

    print("Downloading finished, outcome unknown.")

  except requests.exceptions.RequestException as e:
    print(f"Error downloading files: {e}")



# Initiate download of files
folder_url = f'http://66.213.177.43/v2_{file_type}_files/'
destination_folder = root_dir + f'{file_type}_files'
now = datetime.now(timezone.utc)
start_time = datetime(2025, 6, 3, 16, 38)
end_time = datetime(2025, 6, 3, 17, 40)

download_files(folder_url, destination_folder, start_time, end_time, file_type)

In [16]:
# Load the .npy files into a list
npy_files = sorted(glob(root_dir + 'npy_files/*.npy'))

for i in enumerate(npy_files):
    array = np.load(i[1])
    
    array[array < 20] = 0
    array[array > 80] = 80

    np.save(i[1], array)

In [17]:
import subprocess

print("Finished downloading and data processing at " + datetime.now().strftime('%d/%m/%y %H:%M:%S.%f'))

Finished downloading and data processing at 03/06/25 12:31:31.647273
